In [0]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from numpy import argmax
from scipy.io import loadmat
import time
import math
from datetime import timedelta
from sklearn.preprocessing import OneHotEncoder 

In [0]:
#TENSORFLOW SESSION
sess = tf.InteractiveSession()

In [0]:
#LOAD DATA FROM THE MATRIX FILE

train = loadmat('/content/drive/My Drive/Machine Learning/Lab-1/train_32x32.mat')
test  = loadmat('/content/drive/My Drive/Machine Learning/Lab-1/test_32x32.mat')

X_train = train["X"]
y_train = train["y"]

X_test = test["X"]
y_test = test["y"]

#RESHAPING THE DATA INTO [NUM_IMAGES,ROWS,COLUMNS,CHANNELS] FORMAT
train_images, train_labels = X_train.transpose((3,0,1,2)), y_train[:,0]
test_images, test_labels = X_test.transpose((3,0,1,2)), y_test[:,0]

In [0]:
#CONVERT THE VALUES FROM INT TO FLOAT
train_images = train_images/np.float32(256)
test_images = test_images/np.float32(256)

In [0]:
#Convolutional Layer 1
filter_size1 = 5
num_filters1 = 16

#Convolutional Layer 2
filter_size2 = 5
num_filters2 = 36

#Convolutional Layer 3
filter_size3 = 5
num_filters3 = 64

#Fully Connected Layer
fc_size = 500              #Number of neurons
fc_size2 = 400             #Number of neurons

In [0]:
#CONVERTING LABEL FOR 10 TO 0
train_labels[train_labels==10] = 0
test_labels[test_labels==10] = 0

In [0]:
#CONVERTING THE TRAINING AND TEST LABELS IN ONE HOT ENCODING FORMAT
from sklearn.preprocessing import OneHotEncoder
 
# Fit the OneHotEncoder
enc = OneHotEncoder().fit(y_train.reshape(-1, 1))
enc_test = OneHotEncoder().fit(y_test.reshape(-1, 1))
# Transform the label values to a one-hot-encoding scheme
train_labels_1h = enc.transform(y_train.reshape(-1, 1)).toarray()
test_labels_lh = enc_test.transform(y_test.reshape(-1, 1)).toarray()

In [0]:
#FOR FASTER AND BETTER EXECUTION OF THE CODE, LETS CONVERT THE RGB IMAGES TO GRAYSCALE
def rgb2gray(images):
    return np.expand_dims(np.dot(images, [0.2990, 0.5870, 0.1140]), axis=3)

In [0]:
train_images = rgb2gray(train_images)
test_images = rgb2gray(test_images)

In [0]:
#DATA DIMENSIONS

img_size = 32
num_channels = 1
img_shape = [img_size,img_size,num_channels]
img_size_flat = img_size * img_size * num_channels #Images are stored in 1-D array
num_classes = 10

In [0]:
#FUNCTIONS TO ADD NEW VARIABLES

def new_weights(shape):
  return tf.Variable(tf.truncated_normal(shape,stddev = 0.05))

def new_biases(length):
  return tf.Variable(tf.constant(0.05,shape=[length]))

In [0]:
#FUNCTION FOR A NEW CONVOLUTIONAL LAYER

def new_conv_layer(input,num_input_channels,filter_size,num_filters,use_pooling = True):
  shape = [filter_size,filter_size,num_input_channels,num_filters]
  weights = new_weights(shape=shape)
  biases = new_biases(length=num_filters)
  layer = tf.nn.conv2d(input=input, filter=weights, strides=None,padding='SAME')
  layer += biases
  
  if use_pooling:
    layer = tf.nn.max_pool(value=layer,ksize=[1,2,2,1],strides=None,padding='SAME')
    
  layer = tf.nn.relu(layer)
  
  return layer,weights

In [0]:
#FUNCTION TO FLATTEN THE LAYER

def flatten_layer(layer):
  layer_shape = layer.get_shape()
  num_features = np.array(layer_shape[1:4],dtype = int).prod()
  layer_flat = tf.reshape(layer,[-1,num_features])
  
  return layer_flat,num_features

In [0]:
#FUNCTION FOR THE FULLY CONNECTED LAYER

def new_fc_layer(input,num_inputs,num_outputs,use_relu=True):
  
  weights = new_weights(shape=[num_inputs,num_outputs])
  biases = new_biases(length=num_outputs)
  
  layer = tf.matmul(input,weights) + biases
  
  if use_relu:
    layer = tf.nn.leaky_relu(layer)       #USING THE LEAKY RELU ACTIVATION FUNCTION
    
  return layer

In [0]:
#DEFINING THE PLACEHOLDERS FOR INPUT TRAIN AND TEST DATA

x = tf.placeholder(tf.float32, shape=[None, img_size,img_size,1], name ='x')
x_image = tf.reshape(x,[-1,img_size,img_size,num_channels])


In [0]:
y_true = tf.placeholder(tf.float32, shape=[None,10], name='y_true')
y_true_cls = tf.argmax(y_true,axis=1)

In [0]:
#CREATING THE FIRST CONVOLUTION LAYER

layer_conv1, weights_conv1 = new_conv_layer(input = x_image,num_input_channels=num_channels,filter_size=filter_size1,num_filters=num_filters1,use_pooling=True)

In [0]:
#CREATING THE SECOND CONVOLUTION LAYER

layer_conv2, weights_conv2 = new_conv_layer(input = layer_conv1,num_input_channels=16,filter_size=filter_size2,num_filters=num_filters2,use_pooling=True)

In [0]:
#CREATING THE THIRD CONVOLUTION LAYER

layer_conv3, weights_conv3 = new_conv_layer(input = layer_conv2,num_input_channels=36,filter_size=filter_size3,num_filters=num_filters3,use_pooling=True)

In [0]:
#FLATTEN LAYER

layer_flat,num_features = flatten_layer(layer_conv3)

In [0]:
#FULLY CONNECTED LAYER

layer_fc1 = new_fc_layer(input=layer_flat,num_inputs=num_features,num_outputs=fc_size,use_relu=True)

In [0]:
#FULLY CONNECTED LAYER 2

layer_fc2 = new_fc_layer(input=layer_fc1,num_inputs=fc_size,num_outputs=fc_size2,use_relu=True)

In [0]:
#FULLY CONNECTED LAYER 3

layer_fc3 = new_fc_layer(input=layer_fc2,num_inputs=fc_size2,num_outputs=num_classes,use_relu=False)

In [0]:
#PREDICTED CLASS

y_pred = tf.nn.softmax(layer_fc3)

In [0]:
#ORIGINAL CLASS

y_pred_cls = tf.argmax(y_pred,axis=1) 

In [0]:
#COST FUNCTION TO BE OPTIMIZED

cross_entropy = tf.nn.softmax_cross_entropy_with_logits_v2(logits=layer_fc3,labels=y_true)

In [0]:
cost = tf.reduce_mean(cross_entropy)

In [0]:
#OPTIMIZATION METHOD

optimizer = tf.train.AdamOptimizer(1e-4).minimize(cost)

In [0]:
#PERFORMANCE MEASURES

correct_prediction = tf.equal(y_pred_cls,y_true_cls)
accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))

In [0]:
#FUNCTION FOR RANDOM BATCH SELECTION FROM THE TRAIN DATA SET
def next_batch(num, data, labels):
    '''
    Return a total of `num` random samples and labels. 
    '''
    idx = np.arange(0 , len(data))
    np.random.shuffle(idx)
    idx = idx[:num]
    data_shuffle = [data[ i] for i in idx]
    labels_shuffle = [labels[ i] for i in idx]

    return np.asarray(data_shuffle), np.asarray(labels_shuffle)

In [0]:
#INITIALIZE ALL THE TENSORFLOW VARIABLES

tf.global_variables_initializer().run()

In [0]:
#TRAINING FUNCTION

total_iterations = 0
def optimize(num_iterations):
  
  global total_iterations
  for j in range(total_iterations,total_iterations+num_iterations):
    
    X,y = next_batch(256,train_images,train_labels_1h)
    feed_dict_train = {x: X, y_true: y}
    sess.run(optimizer,feed_dict=feed_dict_train)
    if j % 100 == 0:
      acc = sess.run(accuracy, feed_dict=feed_dict_train)
      msg = "Optimization Iteration: {0:>3}, Training Accuracy: {1:>6.1%}"
      print(msg.format(j, acc))
      
    total_iterations += num_iterations



In [46]:
optimize(501) #501 EPOCHS

Optimization Iteration:   0, Training Accuracy:  22.3%
Optimization Iteration: 100, Training Accuracy:  37.9%
Optimization Iteration: 200, Training Accuracy:  69.1%
Optimization Iteration: 300, Training Accuracy:  80.5%
Optimization Iteration: 400, Training Accuracy:  82.8%
Optimization Iteration: 500, Training Accuracy:  85.2%


In [0]:
#TEST ACCURACY

test_batch_size = 256

def print_test_accuracy():

    num_test = len(test_images)
    cls_pred = np.zeros(shape=num_test, dtype=np.int)
    i = 0
    while i < num_test:
        j = min(i + test_batch_size, num_test)
        images = test_images[i:j, :]
        labels = test_labels_lh[i:j, :]
        feed_dict = {x: images,y_true: labels}
        cls_pred[i:j] = sess.run(y_pred_cls, feed_dict=feed_dict)
        i = j

    cls_true = test_labels
    correct = (cls_true == cls_pred)
    correct_sum = correct.sum()
    acc = float(correct_sum) / num_test

    # Print the accuracy.
    msg = "Accuracy on Test-Set: {0:.1%} ({1} / {2})"
    print(msg.format(acc, correct_sum, num_test))



In [52]:
print_test_accuracy() #TEST ACCURACY AFTER 501 EPOCHS

Accuracy on Test-Set: 84.1% (21893 / 26032)


In [53]:
optimize(201) #FEW MORE OPTIMIZATIONS

Optimization Iteration: 251100, Training Accuracy:  91.8%
Optimization Iteration: 251200, Training Accuracy:  89.1%


In [54]:
print_test_accuracy() #TEST ACCURACY AFTER 702 EPOCHS

Accuracy on Test-Set: 84.7% (22059 / 26032)
